## **OPC Monthly Selection**

*1. Select information from SQL server*

*2. Download CSV data from SQL server*

*3. Upload CSV data here*



*- Press ctrl+f9 to rull all data. Make sure that current team and file name is accurate -*

**List of current team members**

In [ ]:
current_team = ['Lara Crisp', 'Shianne Elliott', 'Conan Cecconie', 'Jonathan Reale', 'Jacob Friedman', 'Taylor Keyes', 'Beau Thomas']

**The name of the uploaded file should EXACTLY match the one below**

In [ ]:
file_name = 'Program Ledger By Personnel.csv'

**The following cell will ask you for a month**

In [ ]:
#@title
import pandas as pd
import numpy as np
import datetime

BGV = pd.read_csv(file_name)
BGV['TourDate1'] = pd.to_datetime(BGV['TourDate1'])
BGV['TourMonth'] = BGV['TourDate1'].dt.strftime('%Y-%m')
print('Please enter month as "yyyy-mm"')

valid=False
while valid == False:
  date = input('Month: ')
  if str(date) in np.sort(BGV['TourMonth'].astype(str).unique()):
    print('Success!')
    valid=True
  else:
    print('Error: This was either typed incorrectly or the month is not in the uploaded file')

BGV = BGV.loc[BGV['TourMonth'] == date]

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

BGV.drop_duplicates(subset=['PersonnelNameTitle', 'TourID2'], inplace=True)

opc_team = pd.DataFrame()
for individual in range(len(current_team)):
  opc_team = opc_team.append(BGV.loc[BGV['PersonnelNameTitle'] == str(current_team[individual])+': Tour Booker'])

BGV = BGV.sort_values('TourDate1')
start_date = BGV['TourDate1'].iat[0]
end_date = BGV['TourDate1'].iat[-1]
daterange = pd.date_range(start = start_date, end = end_date, freq='d')

opc_team_showed = opc_team.loc[opc_team['TourStatus2'] == 'Showed']

**Answer the prompts given below**

In [ ]:
#@title
valid=False
while valid==False:
  print('Would you like to look at:')
  print('1) Numbers')
  print('2) Graphs')
  response = input('')
  if response in ['1', 'numbers', 'Numbers', 'number', 'Number']:
    token = 'numbers'
    valid=True
  elif response in ['2', 'Graphs', 'graphs', 'Graph', 'graph']:
    token = 'graphs'
    valid=True
  else:
    print('Please type either "1" or "2"')

if token=='numbers':
  valid=False
  while valid==False:
    print('What numbers would you like to see?')
    print('1) Tour Count')
    print('2) Shown Rate')
    print('3) Purchase Rate')
    response = input('')
    if response in ['1', 'tour count', 'Tour Count', 'Tour count']:
      token = 'tour count'
      valid=True
    elif response in ['2', 'shown rate', 'Shown Rate', 'Shown rate']:
      token = 'shown rate'
      valid = True
    elif response in ['3', 'purchase rate', 'Purchase Rate', 'Purchase rate']:
      token = 'purchase rate'
      valid = True
    else:
      print('Please type "1", "2", or "3"')
else:
  valid=False
  while valid==False:
    print('What type of graph would you like to see?')
    print('1) One Person')
    print('2) The Team')
    response = input('')
    if response in ['1', 'person', 'Person', 'One Person', 'One person']:
      token = 'individual'
      valid=True
    elif response in ['2', 'team', 'Team', 'the team', 'The team', 'The Team']:
      token = 'team'
      valid=True
    else:
      print('Please type "1" or "2"')

if token == 'tour count':
  for individual in range(len(current_team)):
    name = current_team[individual]
    df = opc_team_showed.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker']
    print(name + ': ' + str(df.shape[0]) + ' tours')

elif token == 'shown rate':
  for individual in range(len(current_team)):
    name = current_team[individual]
    if opc_team.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0] > 0:
      show_percent = opc_team_showed.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0] / opc_team.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0]
      print('{}: {:.1f}% '.format(name, show_percent*100))
    else:
      print('{} did not have any tours'.format(name))

elif token == 'purchase rate':
  for individual in range(len(current_team)):
    name = current_team[individual]
    if opc_team.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0] > 0:
      df = opc_team_showed.loc[(opc_team_showed['CommissionVolume'] > 0) & (opc_team_showed['PersonnelNameTitle'] == name+': Tour Booker')]
      percent_purchase = df.loc[(df['ContractStatus1'] == 'Active\r\nActive') | (df['ContractStatus1'] == 'Suspense\r\nSuspense')].shape[0] / opc_team_showed.loc[opc_team_showed['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0]
      # percent_purchase = opc_team_showed.loc[(opc_team['PersonnelNameTitle'] == name + ': Tour Booker') & 
        # (opc_team['TourResult2'] == 'Sale') & (opc_team['ContractStatus1'] != 'Canceled\r\nRescind')].shape[0] / opc_team_showed.loc[opc_team_showed['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0]
      print('{}: {:.1f}% '.format(name, percent_purchase*100))
    else:
      print('{} did not have any tours'.format(name))

elif token == 'team':
  values = []
  team = []
  for name in current_team:
    count = opc_team_showed.loc[opc_team['PersonnelNameTitle'] == name + ': Tour Booker'].shape[0]
    if count > 0:
      values.append(count)
      team.append(name)
    else:
      print('Cannot find data for '+name)

  df = pd.DataFrame()
  df['Team'] = team
  df['Count'] = values

  df.plot('Team', 'Count', kind='bar', figsize = (18, 8), zorder=3).grid(axis='y', zorder=0, alpha=0.75)

  average = opc_team_showed.shape[0] / len(team)
  plt.axhline(y=average, color='lightcoral', zorder=4)
  plt.legend(['Overall Average: '+str(round(average,2)), 'Tour Count'])

  function = -0.05*max(values)

  for i in range(len(team)):
    plt.text(i, values[i]+function, str(values[i]), ha = 'center', fontsize=18, color='white', zorder=5)

  plt.xticks(fontsize=12, rotation=30)
  plt.xlabel('')
  plt.yticks(fontsize=12)
  plt.title('   OPC Team', weight='bold', fontsize=18, loc='left')
  plt.title(str(date)+' Individual Tour Counts   ', fontsize=18, loc='right')

elif token == 'individual':
  print('Whose graph below would you like to look at?')

  df = BGV.loc[BGV['ProgramName2'] == 'Summit County OPC']
  list = df['PersonnelNameTitle'].unique()
  for i in range(len(list)):
    print('{}) {}'.format(i+1, list[i]))

  valid=False
  while valid==False:
    response = input('Please type one of the numbers above: ')
    if response.isdigit():
      response = int(response)
      if response-1 in range(len(list)):

        fig, ax = plt.subplots(figsize=(15,4))
    
        individual_showed = BGV.loc[(BGV['PersonnelNameTitle'] == list[response-1]) & (BGV['TourStatus2'] == 'Showed')]

        individual_daterange = pd.DataFrame(index=daterange)
        individual_daterange['Tour Count'] = individual_showed['TourDate1'].value_counts()
        individual_daterange['Tour Count'] = individual_daterange['Tour Count'].fillna(0)
        individual_daterange['Day'] = individual_daterange.index.strftime('%d')

        individual_daterange.plot('Day', 'Tour Count', kind='bar', color='lightskyblue', alpha=0.75, ax=ax, zorder=3)
        individual_daterange.rolling(7).mean().plot('Day', 'Tour Count', linewidth=3, color='#1f77b4', ax=ax, zorder=5).grid(axis='y', zorder=0, alpha=0.5)

        average = individual_daterange['Tour Count'].fillna(0).mean()
        plt.axhline(y=average, color='lightcoral', zorder=4)
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        plt.xlabel('')
        plt.xticks(rotation=0)
        plt.title('   '+str(list[response-1])+' (Total: '+str(individual_showed.shape[0])+')', weight='bold', loc='left', fontsize=12)
        plt.title(str(date)+' Tour Count Average   ', loc='right', fontsize=12)
        plt.legend(['7 Day Average', 'Overall Average: '+str(round(average,2)), 'Daily Tour Count'])

        valid2=False
        while valid2==False:
          response2 = input("Would you like to see another person's graph (Yes/No)? ")
          if response2 in ['No', 'no', 'N', 'n']:
            valid=True
            valid2=True
          elif response2 in ['Yes', 'yes', 'Y', 'y']:
              valid2=True
          else:
            print('Please type either "Yes" or "No"')

      else:
        print('This is not one of the numbers above...')
    else:
      print('This is not a number...')
else:
  print('Error')